In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows',None)

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/train.csv')

In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
def get_metadata(df):
    nulls = []
    cols = []
    dtypes = []
    unique = []
    data = {}
    for col, n, dtype in zip(df.columns,df.isnull().sum(),df.dtypes):
        cols.append(col)
        nulls.append(n)
        dtypes.append(dtype)
        unique.append(df[col].nunique())
    data = {"column_name":cols,"data_type":dtypes,"null_count":nulls,"unique_count":unique}
    return pd.DataFrame(data)

In [6]:
df_meta = get_metadata(df)
df_meta = df_meta.sort_values(by='null_count',ascending=False)
df_meta

,column_name,data_type,null_count,unique_count
72,PoolQC,object,1453,3
74,MiscFeature,object,1406,4
6,Alley,object,1369,2
73,Fence,object,1179,4
57,FireplaceQu,object,690,5
3,LotFrontage,float64,259,110
59,GarageYrBlt,float64,81,97
64,GarageCond,object,81,5
58,GarageType,object,81,6
60,GarageFinish,object,81,3


In [7]:
df_nulls = df_meta[df_meta['null_count'] > 0]
df_nulls

,column_name,data_type,null_count,unique_count
72,PoolQC,object,1453,3
74,MiscFeature,object,1406,4
6,Alley,object,1369,2
73,Fence,object,1179,4
57,FireplaceQu,object,690,5
3,LotFrontage,float64,259,110
59,GarageYrBlt,float64,81,97
64,GarageCond,object,81,5
58,GarageType,object,81,6
60,GarageFinish,object,81,3


In [8]:
df_nulls['null_perc'] = (df_nulls['null_count']/len(df))*100
df_nulls

,column_name,data_type,null_count,unique_count,null_perc
72,PoolQC,object,1453,3,99.520548
74,MiscFeature,object,1406,4,96.301370
6,Alley,object,1369,2,93.767123
73,Fence,object,1179,4,80.753425
57,FireplaceQu,object,690,5,47.260274
3,LotFrontage,float64,259,110,17.739726
59,GarageYrBlt,float64,81,97,5.547945
64,GarageCond,object,81,5,5.547945
58,GarageType,object,81,6,5.547945
60,GarageFinish,object,81,3,5.547945


In [9]:
cols_to_remove = df_nulls[df_nulls['null_perc'] >= 10]['column_name'].to_list()
cols_to_remove

['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage']

In [10]:
cols_to_check = list(set(df_nulls['column_name'].to_list()) - set(cols_to_remove))
cols_to_check

['BsmtQual',
 'Electrical',
 'GarageCond',
 'MasVnrType',
 'GarageType',
 'GarageYrBlt',
 'BsmtCond',
 'MasVnrArea',
 'BsmtExposure',
 'GarageQual',
 'BsmtFinType2',
 'GarageFinish',
 'BsmtFinType1']

In [11]:
df.drop(cols_to_remove,axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [12]:
df.isnull().sum()

Id                0
MSSubClass        0
MSZoning          0
LotArea           0
Street            0
LotShape          0
LandContour       0
Utilities         0
LotConfig         0
LandSlope         0
Neighborhood      0
Condition1        0
Condition2        0
BldgType          0
HouseStyle        0
OverallQual       0
OverallCond       0
YearBuilt         0
YearRemodAdd      0
RoofStyle         0
RoofMatl          0
Exterior1st       0
Exterior2nd       0
MasVnrType        8
MasVnrArea        8
ExterQual         0
ExterCond         0
Foundation        0
BsmtQual         37
BsmtCond         37
BsmtExposure     38
BsmtFinType1     37
BsmtFinSF1        0
BsmtFinType2     38
BsmtFinSF2        0
BsmtUnfSF         0
TotalBsmtSF       0
Heating           0
HeatingQC         0
CentralAir        0
Electrical        1
1stFlrSF          0
2ndFlrSF          0
LowQualFinSF      0
GrLivArea         0
BsmtFullBath      0
BsmtHalfBath      0
FullBath          0
HalfBath          0
BedroomAbvGr      0


In [13]:
df.dropna(axis=0,inplace=True)
df.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotArea          0
Street           0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinSF1       0
BsmtFinType2     0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
KitchenQual      0
TotRmsAbvGrd

In [14]:
df.shape

(1338, 75)

In [15]:
df.drop('Id',axis=1,inplace=True)
df.shape

(1338, 74)